# Data exploration and cleaning

In [2]:
import os
import re
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import zscore
import ast

## Import data

### for jupter Notebook

In [3]:
# Ensure the file exists in the current directory or provide the correct path
print("Current Working Directory:", os.getcwd())
file_path = 'data/charting-m-points.csv' 

data = pd.read_csv(file_path, encoding='latin1')
	
print(data.columns)

Current Working Directory: C:\Users\gushi\LTU\TennisStrokePrediction


C:\Users\gushi\AppData\Local\Temp\ipykernel_72268\2372044445.py:5: DtypeWarning: Columns (7,12,24,25,26,31,32,33,34,35,45,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, encoding='latin1')


Index(['match_id', 'Pt', 'Set1', 'Set2', 'Gm1', 'Gm2', 'Pts', 'Gm#', 'TbSet',
       'TB?', 'TBpt', 'Svr', 'Ret', 'Serving', '1st', '2nd', 'Notes',
       '1stNoLet', '2ndNoLet', '1stSV', '2ndSV', '1stNoSV', '2ndNoSV', '1stIn',
       '2ndIn', 'isRally1st', 'isRally2nd', 'Sv1', 'Sv2', 'Rally', 'isAce',
       'isUnret', 'isRallyWinner', 'isForced', 'isUnforced', 'isDouble',
       'rallyNoSpec', 'rallyNoError', 'rallyNoDirection', 'rallyLen',
       'PtWinner', 'isSvrWinner', 'PtsAfter', 'GmW', 'Gm1.1', 'Gm2.1', 'SetW',
       'Set1.1', 'Set2.1', 'RevTB', 'TBrev', 'rallyCount'],
      dtype='object')


## Data cleaning

In [4]:
dropped_features = ["TbSet", "TBpt", "1st", "2nd", "Notes", "1stNoLet", "2ndNoLet", "1stSV", "2ndSV", "1stNoSV", "2ndNoSV", "1stIn", "2ndIn", "isRally1st", "isRally2nd", "Rally", "rallyNoSpec", "rallyNoDirection", "PtWinner", "isSvrWinner", "PtsAfter", 'GmW', 'Gm1.1', 'Gm2.1', 'SetW', 'Set1.1', 'Set2.1', "RevTB", "TBrev", "rallyCount"]
kept_features = ["Pt", "Set1", "Set2", "Gm1", "Gm2", "Pts", "Gm#", "TB?", "rallyLen"]
processing_features = ["match_id", "Svr", "Ret", "Serving", "Sv1", "Sv2", "isAce", "isUnret",
                       "isRallyWinner", "isForced", "isUnforced", "isDouble", "rallyNoError"]

data = data.drop(columns=dropped_features, errors='ignore')
kept_features_data = data[kept_features].copy()
processing_features_data = data[processing_features].copy()
data = pd.concat([kept_features_data, processing_features_data], axis=1)
print("Missing values:")
print(data.isnull().sum())

Missing values:
Pt                    0
Set1                  0
Set2                  0
Gm1                   0
Gm2                   1
Pts                   0
Gm#                   1
TB?                  75
rallyLen              0
match_id              0
Svr                   0
Ret                   0
Serving             946
Sv1                   0
Sv2              205088
isAce                 0
isUnret              10
isRallyWinner        10
isForced             10
isUnforced            0
isDouble              0
rallyNoError      44059
dtype: int64


In [5]:
data = pd.concat([kept_features_data, processing_features_data], axis=1)
print("\nMissing values:")
print(data.isnull().sum())


Missing values:
Pt                    0
Set1                  0
Set2                  0
Gm1                   0
Gm2                   1
Pts                   0
Gm#                   1
TB?                  75
rallyLen              0
match_id              0
Svr                   0
Ret                   0
Serving             946
Sv1                   0
Sv2              205088
isAce                 0
isUnret              10
isRallyWinner        10
isForced             10
isUnforced            0
isDouble              0
rallyNoError      44059
dtype: int64


# Process deduced features and compount features

In [6]:
def filter_data_by_player(data, target_player):
    """
    Filters the data for rows where the target player is playing.

    Parameters:
    data (pd.DataFrame): The input dataset.
    target_player (str): The name of the target player.

    Returns:
    pd.DataFrame: Filtered dataset containing only rows where the target player is playing.
    """
    # Select "match_id" where target player is playing
    selected_match_ids = data.loc[data['Serving'] == target_player, 'match_id'].unique()

    # Filter rows in data where "match_id" is in the selected match_ids
    filtered_data = data[data['match_id'].isin(selected_match_ids)]
    
    return filtered_data

#In processing_data create "Svr" : 1 if the target player is serving, 0 if the target player is receiving
def create_svr_column(data, target_player): 
    """
    Creates a new column "Svr" in the dataset indicating if the target player is serving.

    Parameters:
    data (pd.DataFrame): The input dataset.
    target_player (str): The name of the target player.

    Returns:
    pd.DataFrame: Updated dataset with the new "Svr" column.
    """
    data['Svr'] = np.where(data['Serving'] == target_player, 1, 0)
    return data

def align_score_to_target_perspective(df):
    score_map = {'0': 0, '15': 1, '30': 2, '40': 3, 'AD': 4}
    df[['server_score_raw', 'receiver_score_raw']] = df['Pts'].str.split('-', expand=True)
    df['server_score'] = df['server_score_raw'].map(score_map)
    df['receiver_score'] = df['receiver_score_raw'].map(score_map)

    # Align to target player's perspective
    flip_mask = df['Svr'] == 0
    df.loc[flip_mask, ['server_score', 'receiver_score']] = df.loc[flip_mask, ['receiver_score', 'server_score']].values
    df.loc[flip_mask, ['server_score_raw', 'receiver_score_raw']] = df.loc[flip_mask, ['receiver_score_raw', 'server_score_raw']].values

    df.rename(columns={
        'server_score': 'player_score',
        'receiver_score': 'opponent_score'
    }, inplace=True)

    return df

In [7]:
def enrich_score_features(df):
    df['is_deuce'] = ((df['player_score'] == 3) & (df['opponent_score'] == 3)).astype(int)
    df['is_break_point'] = ((df['opponent_score'] >= 3) & (df['player_score'] < 3)).astype(int)
    df['is_game_point'] = ((df['player_score'] >= 3) & (df['opponent_score'] < 3)).astype(int)
    df['point_diff'] = df['player_score'] - df['opponent_score']
    return df

def enrich_match_context(df):
    df['total_sets_played'] = df['Set1'] + df['Set2']
    df['total_games_played'] = df['Gm1'] + df['Gm2']
    df['is_tiebreak'] = df['TB?'].astype(int)
    df['match_pressure_score'] = (
        df['is_break_point'] + df['is_game_point'] + df['is_tiebreak'] + df['is_deuce']
    )
    return df

def estimate_stamina(df):
    df['rally_intensity'] = df.groupby(['Set1', 'Set2', 'Gm#'])['rallyLen'].transform('mean')
    df['fatigue_index'] = (
        df['total_sets_played'] * 2 +
        df['total_games_played'] +
        df['rallyLen'] / 10 +
        df['is_tiebreak'] * 3
    )
    df['estimated_stamina'] = 1 / (1 + df['fatigue_index'])
    return df

In [8]:
# define shot types to be mapped for our unforced error and winners array
shot_types = [
    'f', 'b',  # groundstrokes
    'r', 's',  # slices
    'v', 'z',  # volleys
    'o', 'p',  # overheads
    'u', 'y',  # drop shots
    'l', 'm',  # lobs
    'h', 'i',  # half-volleys
    'j', 'k',  # swinging volleys
    #'t', 'q'   # trick shots and unknown shots
]

# generate combinations with directions 1, 2, 3
shot_vocab = {f"{shot}{n}": idx for idx, (shot, n) in enumerate(
    (s, i) for s in shot_types for i in [1, 2, 3]
)}


def process_rally_data(df, shot_vocab):
    sequence_data = []
    direction_dict = {'1', '2', '3'}
    serve_dict={'4','5','6'}

    for _, row in df.iterrows():
        isServe = row['Svr'] == 1
        rally = str(row['rallyNoError'])

        if pd.isna(rally) or len(rally) < 2:
            continue

        debug_logs = []
        tokens = []

        # First shot: allow 1–2 digits
        match = re.match(r'^([a-zA-Z])(\d+)', rally)
        if not match:
            continue

        first_letter = match.group(1)
        digits = match.group(2)[:2]
        first_token = first_letter + digits[0]

        if first_token in shot_vocab:
            tokens.append(first_token)
        else:
            continue
        idx = len(match.group(0))

        # Extract remaining shots
        while idx + 2 <= len(rally):
            segment = rally[idx:idx+3]
            debug_logs.append(f"[{idx}] Segment: '{segment}'")

            if re.match(r'^[a-zA-Z]\d[a-zA-Z]$', segment):
                token = segment[0] + segment[1]
                debug_logs.append(f"[{idx}] Pattern A: {segment}")

                if token in shot_vocab:
                    tokens.append(token)

                else:
                    debug_logs.append(f"[{idx}] ❌ Invalid token: {token}")
                    break
                idx += 2

            elif re.match(r'^[a-zA-Z]\d\d$', segment):
                debug_logs.append(f"[{idx}] Pattern B: {segment}")
                token1 = segment[0] + segment[1]
                token2 = segment[0] + segment[2]

                if segment[1] in direction_dict and token1 in shot_vocab:
                    tokens.append(token1)

                elif segment[2] in direction_dict and token2 in shot_vocab:
                    tokens.append(token2)

                else:
                    debug_logs.append(f"[{idx}] ❌ Invalid tokens: {token1}, {token2}")
                    break
                idx += 3

            elif re.match(r'^[a-zA-Z][a-zA-Z]\d$', segment):
                debug_logs.append(f"[{idx}] Pattern C: {segment}")
                token = segment[1] + segment[2]

                if token in shot_vocab:
                    tokens.append(token)

                else:
                    debug_logs.append(f"[{idx}] ❌ Invalid token: {token}")
                    break
                idx += 3

            else:
                debug_logs.append(f"[{idx}] Pattern D (Fallback): {segment}")

                if idx + 1 < len(rally):
                    ch1, ch2 = rally[idx], rally[idx + 1]
                    token = ch1 + ch2

                    if ch1.isalpha() and ch2 in direction_dict and token in shot_vocab:
                        tokens.append(token)
                        idx += 2

                    else:
                        debug_logs.append(f"[{idx}] ❌ Invalid fallback token: {token}")
                        break
                else:
                    break

        # Skip rallies with invalid parsing
        if any(log.startswith("❌") for log in debug_logs):
            print(f"\n🔍 Invalid rally at row {row.name}: {rally}")
            for log in debug_logs:
                print(log)
            continue

        # --- Serve Logic ---
        if not pd.isna(row['Sv1']) and row['Sv1'][0] in serve_dict:
            if not pd.isna(row['Sv2']) and row['Sv2'][0] in serve_dict:
                full_rally = [row['Sv1'][0], row['Sv2'][0]] + tokens
            else:
                full_rally = ['0', row['Sv1'][0]] + tokens
        else:
            continue

        # --- Label Arrays ---
        winner_array = np.zeros(48)
        unforced_array = np.zeros(48)

        final_shot = tokens[-1]
        idx = shot_vocab.get(final_shot, None)
        if idx is not None:
            if row['isRallyWinner']:
                winner_array[idx] += 1
            elif row['isUnforced']:
                unforced_array[idx] += 1

        # --- 3-Shot Sequence Construction ---
        i = 0 if isServe else 1
        while i + 3 < len(full_rally):
            new_row = row.to_dict()
            new_row['shot1'] = full_rally[i]
            new_row['shot2'] = full_rally[i + 1]
            new_row['shot3'] = full_rally[i + 2]
            new_row['shot4'] = full_rally[i + 3]
            new_row['winner_array'] = winner_array.copy()
            new_row['unforced_array'] = unforced_array.copy()
            sequence_data.append(new_row)
            i += 2

    return pd.DataFrame(sequence_data)


In [9]:
def validate_column_types(df):
    expected_types = {
        'Pt': 'int64', 'Set1': 'int64', 'Set2': 'int64', 'Gm1': 'int64',
        'Gm2': 'float64', 'Pts': 'object', 'Gm#': 'object',
        'TB?': 'float64', 'rallyLen': 'int64'
    }
    for col, expected in expected_types.items():
        if col in df.columns and df[col].dtype != expected:
            print(f"Column {col} has type {df[col].dtype}, expected {expected}")
    return df

def validate_score_format(df):
    valid_scores = {'0', '15', '30', '40', 'AD'}
    df[['server_score_raw', 'receiver_score_raw']] = df['Pts'].str.split('-', expand=True)
    invalid_scores = df[
        (~df['server_score_raw'].isin(valid_scores)) |
        (~df['receiver_score_raw'].isin(valid_scores))
    ]
    if not invalid_scores.empty:
        print("Invalid score entries found:")
        print(invalid_scores[['Pts']].drop_duplicates())
    return df[~df.index.isin(invalid_scores.index)]

def validate_set_game_counts(df):
    invalid_sets = df[(df['Set1'] > 3) | (df['Set2'] > 3)]
    invalid_games = df[(df['Gm1'] > 7) | (df['Gm2'] > 7)]
    df = df.drop(invalid_sets.index.union(invalid_games.index))
    return df

def validate_tennis_data(df):
    df = validate_column_types(df)
    df = validate_score_format(df)
    df = validate_set_game_counts(df)
    return df

def preprocess_numeric_columns(df):
    # Clean Gm# to integer
    if 'Gm#' in df.columns:
        df['Gm#'] = df['Gm#'].apply(lambda x: int(re.match(r'\d+', str(x)).group()) if re.match(r'\d+', str(x)) else 0)

    # Ensure score columns are integers where applicable
    score_cols = ['Set1', 'Set2', 'Gm1', 'Gm2']
    for col in score_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Rally length cleanup
    if 'rallyLen' in df.columns:
        df['rallyLen'] = pd.to_numeric(df['rallyLen'], errors='coerce').fillna(0).astype(int)

    return df

def handle_missing_values(df):
    df['TB?'] = df['TB?'].fillna(0)
    df.dropna(subset=['Gm2', 'Gm#'], inplace=True)
    return df

### Acutual processing

In [10]:
def process_tennis_data(data, target_player="RF"):
    df = data.copy()
    df = filter_data_by_player(df, target_player)
    df = create_svr_column(df, target_player)
    df = validate_tennis_data(df)    
    df = align_score_to_target_perspective(df)
    df = process_rally_data(df, shot_vocab=shot_vocab)
    df = enrich_score_features(df)
    df = enrich_match_context(df)
    df = estimate_stamina(df)
    df = handle_missing_values(df)
    df = preprocess_numeric_columns(df)
    df = df.drop(columns=processing_features, errors='ignore')
    return df

processed_data = process_tennis_data(data, target_player="RF")

Invalid score entries found:
          Pts
7108      0-1
7109      1-1
7110      1-2
7111      2-2
7112      2-3
...       ...
266584  16-17
266585  17-17
266586  17-18
266587  18-18
266588  19-18

[77 rows x 1 columns]


In [11]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69077 entries, 0 to 69076
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Pt                    69077 non-null  int64  
 1   Set1                  69077 non-null  int32  
 2   Set2                  69077 non-null  int32  
 3   Gm1                   69077 non-null  int32  
 4   Gm2                   69077 non-null  int32  
 5   Pts                   69077 non-null  object 
 6   Gm#                   69077 non-null  int64  
 7   TB?                   69077 non-null  float64
 8   rallyLen              69077 non-null  int32  
 9   server_score_raw      69077 non-null  object 
 10  receiver_score_raw    69077 non-null  object 
 11  player_score          69077 non-null  float64
 12  opponent_score        69077 non-null  float64
 13  shot1                 69077 non-null  object 
 14  shot2                 69077 non-null  object 
 15  shot3              

In [13]:
print("Unique labels in shot1:", np.unique(processed_data['shot4']))

Unique labels in shot1: ['b1' 'b2' 'b3' 'f1' 'f2' 'f3' 'h1' 'h2' 'h3' 'i1' 'i2' 'i3' 'j1' 'j2'
 'j3' 'k1' 'k3' 'l1' 'l2' 'l3' 'm1' 'm2' 'm3' 'o1' 'o2' 'o3' 'p1' 'p2'
 'p3' 'r1' 'r2' 'r3' 's1' 's2' 's3' 'u1' 'u2' 'u3' 'v1' 'v2' 'v3' 'y1'
 'y2' 'y3' 'z1' 'z2' 'z3']


In [14]:
# Backup dataset
processed_data.to_csv('data/processed_features.csv', index=False)

### quality check

In [114]:
def parse_array_column(df, column):
    return df[column].apply(lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else np.zeros(48))

processed_data['winner_array'] = parse_array_column(processed_data, 'winner_array')
processed_data['unforced_array'] = parse_array_column(processed_data, 'unforced_array')

In [ ]:
def clip_outliers(df, cols, lower=0.01, upper=0.99):
    for col in cols:
        q_low = df[col].quantile(lower)
        q_high = df[col].quantile(upper)
        df[col] = df[col].clip(q_low, q_high)
    return df

numerical_cols = [
    'player_score', 'opponent_score', 'point_diff', 'rallyLen',
    'rally_intensity', 'fatigue_index', 'estimated_stamina',
    'total_sets_played', 'total_games_played', 'match_pressure_score'
]
processed_data = clip_outliers(processed_data, numerical_cols)

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

shot_columns = ['shot1', 'shot2', 'shot3', 'shot4']
label_encoder = LabelEncoder()
for col in shot_columns:
    processed_data[col + '_encoded'] = label_encoder.fit_transform(processed_data[col])

scaler = StandardScaler()
processed_data[numerical_cols] = scaler.fit_transform(processed_data[numerical_cols])

In [ ]:
binary_cols = ['is_deuce', 'is_break_point', 'is_game_point', 'is_tiebreak']

X_seq = []
features = processed_data[numerical_cols + binary_cols].values
shot1 = processed_data['shot1_encoded'].values
shot2 = processed_data['shot2_encoded'].values
y = processed_data['target_class'].values

for f, s1, s2 in zip(features, shot1, shot2):
    t1 = np.concatenate(([s1], f))
    t2 = np.concatenate(([s2], f))
    X_seq.append([t1, t2])

X_seq = np.array(X_seq)  # shape: (samples, 2, input_size)
print("RNN input shape:", X_seq.shape)

In [118]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
import pandas as pd


# Copy the data
processed_data_xgb = processed_data.copy()

# xgboost only accepts numerical and categorical data, so we split the 54 size arrays into one column each
winner_df = pd.DataFrame(processed_data_xgb['winner_array'].tolist(), index=processed_data_xgb.index)
winner_df.columns = [f'winner_{i}' for i in range(winner_df.shape[1])]

unforced_df = pd.DataFrame(processed_data_xgb['unforced_array'].tolist(), index=processed_data_xgb.index)
unforced_df.columns = [f'unforced_{i}' for i in range(unforced_df.shape[1])]

# Drop the original object columns
processed_data_xgb.drop(columns=['winner_array', 'unforced_array'], inplace=True)

# Concatenate the unpacked columns
processed_data_xgb = pd.concat([processed_data_xgb, winner_df, unforced_df], axis=1)

# Step 1: Separate features and target BEFORE encoding
X = processed_data_xgb.drop(columns=['shot4'])
y_raw = processed_data_xgb['shot4']

# Step 2: Train-test split
X_train, X_test, y_train_raw, y_test_raw = train_test_split(X, y_raw, test_size=0.2, random_state=42)

# Step 3: Encode categorical feature columns
categorical_cols = ['Gm#', 'shot1', 'shot2', 'shot3']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    all_values = pd.concat([X_train[col], X_test[col]]).astype(str)
    le.fit(all_values)
    X_train[col] = le.transform(X_train[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))
    label_encoders[col] = le

# Step 4: Encode target column (shot3)
le_target = LabelEncoder()
le_target.fit(pd.concat([y_train_raw, y_test_raw]))
y_train = le_target.transform(y_train_raw)
y_test = le_target.transform(y_test_raw)

X_train_np = np.array(X_train)
y_train_np = np.array(y_train, dtype=np.int32)

missing_classes = set(range(48)) - set(np.unique(y_train_np))
for cls in missing_classes:
    dummy_input = X_train_np[0]  # Just reusing one sample (safe for XGBoost)
    X_train_np = np.vstack([X_train_np, dummy_input])
    y_train_np = np.append(y_train_np, cls)

# Shuffle to avoid the dummy samples sitting at the end
X_train_np, y_train_np = shuffle(X_train_np, y_train_np, random_state=42)

# Step 5: Train model
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', objective='multi:softprob',
    num_class=48)
model.fit(X_train_np, y_train_np)

# Step 6: Evaluate
print("Train Accuracy:", model.score(X_train, y_train))
print("Test Accuracy:", model.score(X_test, y_test))

# Step 1: Get column names
feature_names = X.columns.to_numpy()  # shape: (109,)

# Step 2: Apply importance mask to columns
importances = model.feature_importances_
important_features = feature_names[importances > 0.01]
print("Important features:", important_features)

sorted_indices = np.argsort(importances)[::-1]
sorted_features = feature_names[sorted_indices]
sorted_importances = importances[sorted_indices]

# Top 10 most important
for feat, imp in zip(sorted_features[:20], sorted_importances[:20]):
    print(f"{feat}: {imp:.4f}")


c:\Users\nicho\anaconda3\envs\diffusion-env\lib\site-packages\xgboost\core.py:158: UserWarning: [09:58:07] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Train Accuracy: 0.5614722819676681
Test Accuracy: 0.40525914105321587
Important features: ['rallyLen' 'shot3' 'winner_0' 'winner_1' 'winner_2' 'winner_3' 'winner_4'
 'winner_5' 'winner_12' 'winner_13' 'winner_14' 'winner_15' 'winner_17'
 'winner_18' 'winner_20' 'winner_24' 'winner_26' 'winner_27' 'winner_38'
 'winner_42' 'winner_43' 'winner_44' 'unforced_0' 'unforced_1'
 'unforced_2' 'unforced_3' 'unforced_4' 'unforced_5' 'unforced_9'
 'unforced_10' 'unforced_11' 'unforced_12' 'unforced_13' 'unforced_14'
 'unforced_16' 'unforced_17' 'unforced_26' 'unforced_27' 'unforced_29']
unforced_0: 0.0386
winner_2: 0.0384
winner_0: 0.0317
unforced_5: 0.0316
unforced_1: 0.0310
unforced_4: 0.0309
winner_27: 0.0263
shot3: 0.0250
unforced_27: 0.0246
unforced_17: 0.0230
unforced_2: 0.0225
winner_17: 0.0212
winner_44: 0.0210
winner_3: 0.0198
rallyLen: 0.0189
winner_12: 0.0179
winner_24: 0.0178
winner_15: 0.0177
winner_26: 0.0173
unforced_3: 0.0173


In [90]:
# predict top-3 accuracy score

from sklearn.metrics import top_k_accuracy_score

# Get full list of labels from the label encoder
all_classes = le_target.transform(le_target.classes_)

# compute top-3 accuracy
top3_acc = top_k_accuracy_score(y_test, model.predict_proba(X_test), k=3, labels=all_classes)
print("Top-3 Accuracy:", top3_acc)

Top-3 Accuracy: 0.7328800388538125
